In [61]:
# coding = utf-8
from preprocess import Preprocess
from preprocess import Vectorization

from bm25f_ext import *
import numpy as np
import pandas as pd
DATA_PATH = '../../data/spark.csv'

In [12]:
df = pd.read_csv(DATA_PATH, encoding = 'GB18030')
df['Duplicate_null'] = df['Duplicated_issue'].apply(lambda x : pd.isnull(x))
df_data = df[df['Duplicate_null'] == False]
df_data_issues = df_data[['Issue_id', 'Duplicated_issue']]

In [52]:
df_data_issues.head()

,Issue_id,Duplicated_issue
246,SPARK-533,SPARK-736
258,SPARK-545,SPARK-983
307,SPARK-594,SPARK-612
325,SPARK-612,SPARK-594
333,SPARK-620,SPARK-671


In [59]:
df_data_issues['Duplicated_issue'][df_data_issues[df_data_issues['Issue_id'] == 'SPARK-545'].index].values[0]

u'SPARK-983'

In [14]:
def get_duplist(str1):
    lst = str1.split(';')
    lst = [i for i in lst if i.startswith('SPARK')]
    return lst

In [51]:
def illegal(list_):
    bool_vec = [i.startswiths('SPARK') for i in list_]
    ans = 1
    for i in bool_vec:
        ans = ans & i
    return ans

def generate_training_set(data):    
    total_dup_issues = np.array(pd.concat([data['Issue_id'], data['Duplicated_issue']]))
    print(len(total_dup_issues))
    dup_issues = []
    for i in total_dup_issues:
        for j in get_duplist(i):
            dup_issues.append(j)

    dup_issues = pd.DataFrame(dup_issues).drop_duplicates()
    dup_issues = [i[0] for i in dup_issues.values]
    dup_issues = sorted(dup_issues, reverse = False, key = lambda x: int(x[6:]) )
    print(len(dup_issues))
    
    # step1 generate buckets
    buckets = {}
    random_pool = np.array(data['Issue_id'])
    for i,r in data.iterrows():
        buckets[r['Issue_id']] = get_duplist(r['Duplicated_issue'])
    
    # step2 generate training set
    
    training_set = []
    for k,v in buckets.iteritems():
        for i in v:
            training_set.append([k, i, np.random.choice(dup_issues)])
    training_set = [i for i in training_set if illegel(i)]
    return buckets, training_set

In [49]:
generate_training_set(df_data_issues)

5612
2808
[u'SPARK-533', u'SPARK-545', u'SPARK-594', u'SPARK-612', u'SPARK-620']
[[u'SPARK-533'], [u'SPARK-545'], [u'SPARK-594'], [u'SPARK-612'], [u'SPARK-620'], [u'SPARK-636'], [u'SPARK-650'], [u'SPARK-655'], [u'SPARK-671'], [u'SPARK-672']]


In [13]:
# coding = utf-8
from preprocess import Preprocess
from preprocess import Vectorization
import numpy as np
import pandas as pd
from bm25f_ext import *


DATA_PATH = '../../data/spark.csv'


def unigrams(query):
    '''
        Under construction
    '''
    return

def bigrams(query):
    '''
        Under construction
    '''
    return



def construct_features(d,q, corpus):
    '''
        each query is a dict, keys are fields
        for a query and a document
        construct 7 features
    '''

    d_unigram = d
    q_unigram = q
    feature1 = bm25f_ext(d_unigram, q_unigram, corpus)
    
    # d_bigram = bigrams(d)
    # q_bigram = bigrams(q)
    # feature2 = bm25f_ext(d_bigram, q_bigram)

    # feature3 = 1 if d['product'] == q['product'] else 0
    feature4 = 1 if d['component'] == q['component'] else 0
    
    # feature5 = 1 if d['type'] == q['type'] else 0
    # feature6 = 1 / (1.0 + d['priority'] - q['priority'])
    # feature7 = 1 / (1.0 + d['version'] - q['version'])

    return [feature1, feature4]#, feature4]


def REP(d,q, corpus):
    features = construct_features(d,q, corpus)
    weights = [0.9, 0.2]#, 0.032] # initialize
    ans = 0.0
    for i in range(len(weights)):
        ans += (weights[i] + features[i])

    return ans


def get_duplist(str1):
    lst = str1.split(';')
    lst = [i for i in lst if i.startswith('SPARK')]
    return lst

def illegal(list_):
    bool_vec = [str(i).startswith('SPARK') for i in list_]
    ans = True
    for i in bool_vec:
        ans = ans & i
    return ans
    
def generate_data_set(data):    
    total_dup_issues = np.array(pd.concat([data['Issue_id'], data['Duplicated_issue']]))
    print(len(total_dup_issues))
    dup_issues = []
    for i in total_dup_issues:
        for j in get_duplist(i):
            dup_issues.append(j)

    dup_issues = pd.DataFrame(dup_issues).drop_duplicates()
    dup_issues = [i[0] for i in dup_issues.values]
    dup_issues = sorted(dup_issues, reverse = False, key = lambda x: int(x[6:]) )
    print(len(dup_issues))
    
    # step1 generate buckets
    buckets = {}
    random_pool = np.array(data['Issue_id'])
    for i,r in data.iterrows():
        buckets[r['Issue_id']] = get_duplist(r['Duplicated_issue'])
    
    # step2 generate data set
    data_set = []
    for k,v in buckets.iteritems():
        for i in v:
            data_set.append([k, i, np.random.choice(dup_issues)])
    print(data_set[0])
    data_set = [i for i in data_set if illegal(i)]
    print(len(data_set))
    return buckets, data_set

In [2]:
    preprocess = Preprocess()
    # vectorization = Vectorization()

    df = pd.read_csv(DATA_PATH, encoding = 'GB18030')
    df['Duplicate_null'] = df['Duplicated_issue'].apply(lambda x : pd.isnull(x))
    df_data = df[df['Duplicate_null'] == False]
    df_data['summary_list'] = df_data['Title'].apply(lambda x : preprocess.stem_and_stop_removal(x))
    df_data['desc_list'] = df_data['Description'].apply(lambda x : preprocess.stem_and_stop_removal(str(x)))
    df_data_issues = df_data[['Issue_id', 'Duplicated_issue']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
buckets, data_set = generate_data_set(df_data_issues)

5612
2808
[u'SPARK-18270', u'SPARK-16472', u'SPARK-16901']
3177


In [4]:
training_set = data_set[:int(len(data_set)*0.7)]
test_set = data_set[int(0.7*len(data_set)):int(0.8*len(data_set))]

In [14]:
print(len(data_set))

0


In [11]:
    # unigram:
    training_data = []
    for q in training_set[:5]:
        tmp = []
        for j in q:
            tmp_dict = {}
            smry = df_data['summary_list'][df_data[df_data['Issue_id'] == j].index].values[0]
            tmp_dict['summary'] = smry
            desc = df_data['desc_list'][df_data[df_data['Issue_id'] == j].index].values[0]
            tmp_dict['description'] = desc
            tmp_dict['component'] = df_data['Component'][df_data[df_data['Issue_id'] == j].index].values[0]
            tmp.append(tmp_dict)

        training_data.append(tmp)

In [12]:
corpus = []
for j in df_data['desc_list'].values:
    corpus += j
for j in df_data['summary_list'].values:
    corpus += j
corpus = list(set(corpus))

In [21]:
import math
def RNC(training_data, corpus):
    front = REP(training_data[0], training_data[1], corpus)
    rare = REP(training_data[1], training_data[2], corpus)
    Y = front - rare
    return math.log(1 + math.exp(Y))


In [11]:
def generate_bigram(a):
    bi = []
    for i in range(len(a)-1):
        bi.append(a[i]+a[i+1])
    return bi

a = [12,3]
print generate_bigram(a)

[3, 5]


In [4]:
import pandas as pd
b =[[1, [2,3,4]]]
pd.DataFrame(b)

,0,1
0,1,"[2, 3, 4]"
